In [21]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import accuracy_score, classification_report
# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
# import torch
# from torch.utils.data import DataLoader, Dataset
# import os

# # Load your CSV dataset
# df = pd.read_csv('/content/benchmark.csv')

# # Flatten the CSV into a list of data points
# data_points = []
# for col in df.columns:
#     for value in df[col]:
#         data_points.append(f"{col}: {value}")

# # Create labels based on column names
# labels = [col for col in df.columns for _ in range(len(df))]

# # Encode labels using LabelEncoder
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(labels)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(data_points, y_encoded, test_size=0.2, random_state=42)

# # Tokenize the data with padding token
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# X_train_tokens = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt', max_length=50)
# X_test_tokens = tokenizer(X_test, padding=True, truncation=True, return_tensors='pt', max_length=50)

# # Create a custom dataset
# class CustomDataset(Dataset):
#     def __init__(self, tokens, labels):
#         self.tokens = tokens
#         self.labels = labels

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
#         return {
#             'input_ids': self.tokens['input_ids'][idx],
#             'attention_mask': self.tokens['attention_mask'][idx],
#             'labels': torch.tensor(self.labels[idx])
#         }

# train_dataset = CustomDataset(X_train_tokens, y_train)
# test_dataset = CustomDataset(X_test_tokens, y_test)

# # Set up DataLoader
# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# # Load pre-trained DistilBERT model for sequence classification
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

# # Set up optimizer and loss function using AdamW
# optimizer = AdamW(model.parameters(), lr=5e-5)

# # Training loop
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# num_epochs = 3

# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0

#     for batch in train_loader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         optimizer.zero_grad()
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()

#         loss.backward()
#         optimizer.step()

#     average_loss = total_loss / len(train_loader)
#     print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}')

# # Save the trained model
# model_save_path = r'C:/Workspaces/CodeSpaces/Python_Work/Langchain_Model/saved_model'
# os.makedirs(model_save_path, exist_ok=True)
# model.save_pretrained(model_save_path)

# # Evaluation
# model.eval()
# all_preds = []
# all_labels = []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids, attention_mask=attention_mask)
#         logits = outputs.logits
#         preds = torch.argmax(logits, dim=1)

#         all_preds.extend(preds.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())

# # Decode predictions
# decoded_preds = label_encoder.inverse_transform(all_preds)

# # Evaluate the model
# accuracy = accuracy_score(y_test, all_preds)
# classification_rep = classification_report(y_test, all_preds)

# # Display the results
# print(classification_rep)
# print(accuracy)


import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, Dataset
import os
import numpy as np

# Load your CSV dataset
df = pd.read_csv('/content/benchmark.csv')

# Flatten the CSV into a list of data points
data_points = []
for col in df.columns:
    for value in df[col]:
        data_points.append(f"{col}: {value}")

# Create labels based on column names
labels = [col for col in df.columns for _ in range(len(df))]

# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

label_encoder_file = '/content/label_encoder.npy'
np.save(label_encoder_file, label_encoder.classes_)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_points, y_encoded, test_size=0.2, random_state=42)

# Tokenize the data with padding token
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
X_train_tokens = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt', max_length=50)
X_test_tokens = tokenizer(X_test, padding=True, truncation=True, return_tensors='pt', max_length=50)

# Create a custom dataset
class CustomDataset(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.tokens['input_ids'][idx],
            'attention_mask': self.tokens['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = CustomDataset(X_train_tokens, y_train)
test_dataset = CustomDataset(X_test_tokens, y_test)

# Set up DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Load pre-trained DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

# Set up optimizer and loss function using AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}')

# Save the trained model and tokenizer
model_save_path = '/content/saved_model'
os.makedirs(model_save_path, exist_ok=True)
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Decode predictions
decoded_preds = label_encoder.inverse_transform(all_preds)

# Evaluate the model
accuracy = accuracy_score(y_test, all_preds)
classification_rep = classification_report(y_test, all_preds)

# Display the results
print(classification_rep)
print(accuracy)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Loss: 1.3422716387362659
Epoch 2/3, Loss: 0.020268445155713435
Epoch 3/3, Loss: 0.004971795022535005
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00         6
           3       1.00      1.00      1.00        11
           4       1.00      1.00      1.00         8
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         9
           7       1.00      1.00      1.00         7
           8       1.00      1.00      1.00        12
           9       1.00      1.00      1.00        11
          10       1.00      1.00      1.00        12
          11       1.00      1.00      1.00         8
          12       1.00      1.00      1.00        15
          13       1.00      1.00      1.00         9
          14       1.00      1.00      1.00        11
          15       1.00

In [47]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [49]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, Dataset
import os
import numpy as np

# Load your CSV dataset
df = pd.read_csv('/content/benchmark.csv')

# Flatten the CSV into a list of data points
data_points = []
for index, row in df.iterrows():
    for col, value in zip(df.columns, row):
        data_points.append(f"{col}: {value}")

# Create labels based on column names
labels = [col for col in df.columns for _ in range(len(df))]

# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

label_encoder_file = '/content/label_encoder.npy'
np.save(label_encoder_file, label_encoder.classes_)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_points, y_encoded, test_size=0.2, random_state=42)

# Tokenize the data with padding token
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
X_train_tokens = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt', max_length=50)
X_test_tokens = tokenizer(X_test, padding=True, truncation=True, return_tensors='pt', max_length=50)

# Create a custom dataset
class CustomDataset(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.tokens['input_ids'][idx],
            'attention_mask': self.tokens['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = CustomDataset(X_train_tokens, y_train)
test_dataset = CustomDataset(X_test_tokens, y_test)

# Set up DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Load pre-trained DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

# Set up optimizer and loss function using AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
num_epochs = 50

# Before the training loop
accumulation_steps = 4  # Accumulate gradients over 4 batches

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for i, batch in enumerate(train_loader):
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      optimizer.zero_grad()
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs.loss

      # Perform gradient accumulation
      loss.backward()
      if (i + 1) % accumulation_steps == 0 or i == len(train_loader) - 1:
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Adjust max_norm as needed
          optimizer.step()
          optimizer.zero_grad()

      total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}')

# After the training loop
torch.cuda.empty_cache()

# Save the trained model and tokenizer
model_save_path = '/content/Model/saved_model'
os.makedirs(model_save_path, exist_ok=True)
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Decode predictions
decoded_preds = label_encoder.inverse_transform(all_preds)

# Evaluate the model
accuracy = accuracy_score(y_test, all_preds)
classification_rep = classification_report(y_test, all_preds)

# Display the results
print(classification_rep)
print(accuracy)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: ignored

In [33]:
# import torch
# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder
# import numpy as np  # Import NumPy for loading the label encoder

# # Load the trained model and tokenizer
# model_path = '/content/saved_model'
# model = DistilBertForSequenceClassification.from_pretrained(model_path)
# tokenizer = DistilBertTokenizer.from_pretrained(model_path)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# # Load your CSV dataset for inference
# df_inference = pd.read_csv('/content/AscotBDX08092Main.csv')  # Replace with the path to your inference dataset

# # Load the label encoder from the saved file
# label_encoder_file = '/content/label_encoder.npy'
# label_encoder = LabelEncoder()
# label_encoder.classes_ = np.load(label_encoder_file)

# # Tokenize the new data
# max_batch_size = 2  # Adjust this based on your available GPU memory
# start_idx = 0

# # Store the predictions in a list
# predictions = []

# while start_idx < len(df_inference.columns):
#     end_idx = start_idx + max_batch_size
#     batch_data = list(df_inference.columns[start_idx:end_idx])

#     new_data_tokens = tokenizer(batch_data, padding=True, truncation=True, return_tensors='pt', max_length=50).to(device)

#     # Make predictions
#     model.eval()
#     with torch.no_grad():
#         # Move data to GPU if available
#         new_data_tokens = {key: value.to(device) for key, value in new_data_tokens.items()}

#         outputs = model(**new_data_tokens)
#         logits = outputs.logits
#         preds = torch.argmax(logits, dim=1)

#     # Decode predictions
#     decoded_preds = label_encoder.inverse_transform(preds.cpu().numpy())

#     # Store the predictions in the list
#     predictions.extend(zip(batch_data, decoded_preds))

#     # Clear GPU cache
#     torch.cuda.empty_cache()

#     start_idx = end_idx

# # Display predictions
# print("Predicted Mapping:", predictions)


import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np  # Import NumPy for loading the label encoder

# Load the trained model and tokenizer
model_path = '/content/saved_model'
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Load your CSV dataset for inference
df_inference = pd.read_csv('/content/AscotBDX08092Main.csv')  # Replace with the path to your inference dataset

# Assuming your data is in the same format as during training
data_points_inference = []
column_predictions = []

for col in df_inference.columns:
    col_predictions = {"inference_column": col, "predicted_trained_column": []}
    for value in df_inference[col]:
        data_point = f"{col}: {value}"
        data_points_inference.append(data_point)

        # Tokenize the new data
        new_data_tokens = tokenizer(data_point, padding=True, truncation=True, return_tensors='pt', max_length=50).to(device)

        # Make predictions
        model.eval()
        with torch.no_grad():
            # Move data to GPU if available
            new_data_tokens = {key: value.to(device) for key, value in new_data_tokens.items()}
            outputs = model(**new_data_tokens)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

        # Decode predictions
        decoded_preds = label_encoder.inverse_transform(preds.cpu().numpy())
        col_predictions["predicted_trained_column"].append(decoded_preds[0])

        # Clear GPU cache
        torch.cuda.empty_cache()

    column_predictions.append(col_predictions)

# Display predictions
for prediction in column_predictions:
    print(prediction)



KeyboardInterrupt: ignored

In [37]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np  # Import NumPy for loading the label encoder

# Load the trained model and tokenizer
model_path = '/content/Model/saved_model'
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Load your CSV dataset for inference
df_inference = pd.read_csv('/content/AscotBDX08092Main.csv')  # Replace with the path to your inference dataset

# Load the label encoder from the saved file
label_encoder_file = '/content/label_encoder.npy'
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load(label_encoder_file)

# Tokenize the new data
max_batch_size = 2  # Adjust this based on your available GPU memory
start_idx = 0

# Initialize a set to keep track of processed columns
processed_columns = set()

# Initialize a list to capture the mapping information
mapping_info = []

while start_idx < len(df_inference.columns):
    end_idx = start_idx + max_batch_size
    batch_data = [": ".join([df_inference.columns[i], str(df_inference.iloc[0, i])]) for i in range(start_idx, end_idx)]

    new_data_tokens = tokenizer(batch_data, padding=True, truncation=True, return_tensors='pt', max_length=50).to(device)

    # Make predictions
    model.eval()
    with torch.no_grad():
        # Move data to GPU if available
        new_data_tokens = {key: value.to(device) for key, value in new_data_tokens.items()}

        outputs = model(**new_data_tokens)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

    # Decode predictions
    decoded_preds = label_encoder.inverse_transform(preds.cpu().numpy())

    # Capture the mapping information for each unique column
    for i, column_name in enumerate(batch_data):
        if column_name not in processed_columns:
            mapping_info.append({f"Column from Inference CSV": column_name, "Predicted Class Name": decoded_preds[i]})
            processed_columns.add(column_name)

    # Clear GPU cache
    torch.cuda.empty_cache()

    start_idx = end_idx

# Display the final mapping information
print(mapping_info)


[{'Column from Inference CSV': 'uploadid: 497', 'Predicted Class Name': 'Contents Value'}, {'Column from Inference CSV': 'reporting_period: 92023', 'Predicted Class Name': 'Square Footage'}, {'Column from Inference CSV': 'policy_number: ASC-2977825-00', 'Predicted Class Name': 'Full policy number'}, {'Column from Inference CSV': 'expiring_policy_number: nan', 'Predicted Class Name': 'Expiring Policy Number'}, {'Column from Inference CSV': 'policy_effective_date: 05-09-2023', 'Predicted Class Name': 'Policy Effective Date'}, {'Column from Inference CSV': 'policy_expiration_date: 05-09-2024', 'Predicted Class Name': 'Policy Expiration Date'}, {'Column from Inference CSV': 'transaction_effective_date: 01-09-2023', 'Predicted Class Name': 'Transaction Effective Date'}, {'Column from Inference CSV': 'transaction_expiration_date: 05-09-2024', 'Predicted Class Name': 'Transaction Expiration Date'}, {'Column from Inference CSV': 'transaction_type: NB', 'Predicted Class Name': 'Transaction Type